# Players Comparison

This notebook allows you to compare statistics between two NHL players.

## Get players stats

In [ ]:
#! /home/db/.pyenv/shims/python

import requests
import pandas as pd
from ipywidgets import widgets, VBox, HBox, interactive_output
from IPython.display import display, HTML
import json

# Base server URL to get nhl stats
BASE_URL = "http://localhost:8080"

# Initialize variables to store player data
player_a_data = None
player_b_data = None
# Requires a user input
player_a_name_search_pattern = "Connor McDavid"
player_b_name_search_pattern = "Doughty"

player_a_info = None
player_a_name = None
player_b_id = None
player_b_name = None

def get_player_data(player_id):
    """Get player landing data by id

    Args:
        player_id (int)

    Returns:
        _type_: json-encoded content with player landing info
    """
    if player_id:
        response_stats = requests.get(f"{BASE_URL}/player/{player_id}/stats")
        if response_stats.status_code == 200:
            return response_stats.json()

def get_player_info(player_search_pattern):
    """Get player id by a search pattern

    Args:
        player_search_pattern (str): Player name to search for

    Returns:
        _type_: (int, str)
    """
    if player_search_pattern:
        response = requests.get(f"{BASE_URL}/player", params={"q": player_search_pattern})
        if response.status_code == 200:
            result = response.json()
            if isinstance(result, list) and len(result) > 0:
                player_id = result[0]['playerId']
                player_name = result[0]['name']
                other_possible_options = [f"\n{p['name']} (ID: {p['playerId']})" for p in result[1:]]
                print(f"✓ Found Player: {result[0]['name']} (ID: {player_id})")
                if other_possible_options:
                    print(f"Other possible players: {', '.join(other_possible_options)}")
                return (player_id, player_name)
            else:
                print(f"✗ Player not found")
        else:
            print(f"✗ Error searching Player {player_search_pattern}: {response.status_code}")
            
try:
    player_a_info = get_player_info(player_a_name_search_pattern)
    player_b_info = get_player_info(player_b_name_search_pattern)
    
    player_a_id = player_a_info[0]
    player_b_id = player_b_info[0]
    
    player_a_name = player_a_info[1]
    player_b_name = player_b_info[1]
    
    player_a_data = get_player_data(player_a_id)
    player_b_data = get_player_data(player_b_id)
except Exception as e:
    print(f"Error: {str(e)}")

✓ Found Player: Connor McDavid (ID: 8478402)
Other possible players: 
Connor Trenholm (ID: 8483048), 
Connor Lacouvee (ID: 8481346), 
Connor McMichael (ID: 8481580), 
Connor Murphy (ID: 8484121), 
Connor Crisp (ID: 8477398), 
Connor Hardowa (ID: 8477241), 
Connor Bedard (ID: 8484144), 
Connor Stokes (ID: 8476571), 
Connor Wilson (ID: 8483154), 
Kenny Connors (ID: 8483675), 
Connor Dewar (ID: 8480980), 
Connor Mylymok (ID: 8485166), 
Connor Knapp (ID: 8475303), 
Connor MacEachern (ID: 8484283), 
Connor Kurth (ID: 8483727), 
Connor Hicks (ID: 8479642), 
Connor Kelley (ID: 8482948), 
Connor Mayer (ID: 8484897), 
Connor McClennon (ID: 8482220)
--- (8478402, 'Connor McDavid')
✓ Found Player: Drew Doughty (ID: 8474563)
player_a_name: Connor McDavid
player_b_name: Drew Doughty


## Season Totals Comparison

In [ ]:
season = widgets.Dropdown(
    options=[],
    description='Season:',
    style={'description_width': '150px'}
)

game_type_id = widgets.Dropdown(
    options=[("Regular Season", 2), ("Playoffs", 3)],
    value=2,
    description='Game Type:',
    style={'description_width': '150px'}
)

display_season_button = widgets.Button(
    description='Display Season Totals',
    button_style='success'
)

def update_season_dropdowns():
    """
    Update season dropdowns by iterating over both player season totals;
    Then merge unique seasons into a single list for the dropdown options;
    Then sort the seasons and select the most recent season by default.
    """
    seasons = set()
    if player_a_data and 'seasonTotals' in player_a_data:
        seasons.update([s['season'] for s in player_a_data['seasonTotals'] if s['season']])
    if player_b_data and 'seasonTotals' in player_b_data:
        seasons.update([s['season'] for s in player_b_data['seasonTotals'] if s['season']])
    
    sorted_seasons = sorted(seasons)
    season.options = [(str(s), s) for s in sorted_seasons]
    if sorted_seasons:
        # Select most recent season
        season.value = sorted_seasons[-1]

def get_season_stats(player_data, season, game_type_id):
    if not player_data or 'seasonTotals' not in player_data:
        return None
    for stat in player_data['seasonTotals']:
        if stat['season'] == season and stat['gameTypeId'] == game_type_id:
            return stat
    return None

def display_season_totals():
    if not player_a_data and not player_b_data:
        print("Please search for players first")
        return
    
    player_a_season_totals = get_season_stats(player_a_data, season.value, game_type_id.value)
    player_b_season_totals = get_season_stats(player_b_data, season.value, game_type_id.value)
    
    # Create comparison table
    comparison_data = []
    
    if player_a_season_totals or player_b_season_totals:
        stat_fields = ['gamesPlayed', 'goals', 'assists', 'points', 'pim', 
                      'powerPlayGoals', 'shorthandedGoals', 'gameWinningGoals', 'shots']
        
        for field in stat_fields:
            row = {'Metric': field}
            if player_a_season_totals:
                row[f"{player_a_name}"] = player_a_season_totals.get(field, '-')
            if player_b_season_totals:
                row[f"{player_b_name}"] = player_b_season_totals.get(field, '-')
            comparison_data.append(row)
        
        df_comparison = pd.DataFrame(comparison_data)
        # display(df_comparison.to_html(escape=False, index=False), HTML('<br>'))
        display(df_comparison)
    else:
        print("No season stats found for the selected criteria")

update_season_dropdowns()

season_output = widgets.Output()

def on_display_season_click(_):
    season_output.clear_output()
    with season_output:
        display_season_totals()

display_season_button.on_click(on_display_season_click)

display(
    VBox([
        HBox([season, game_type_id]),
        display_season_button,
        season_output
    ])
)

## Aggregated Season Totals (All Seasons)

In [ ]:
# def display_aggregated_season_totals():
#     if not player_a_data and not player_b_data:
#         print("Please search for players first.")
#         return
    
#     # Helper function to aggregate stats
#     def aggregate_season_stats(player_data, game_type_id):
#         if not player_data or 'seasonTotals' not in player_data:
#             return None
        
#         stats = [s for s in player_data['seasonTotals'] if s['gameTypeId'] == game_type_id]
#         if not stats:
#             return None
        
#         aggregated = {
#             'gamesPlayed': sum(s.get('gamesPlayed', 0) for s in stats),
#             'goals': sum(s.get('goals', 0) for s in stats),
#             'assists': sum(s.get('assists', 0) for s in stats),
#             'points': sum(s.get('points', 0) for s in stats),
#             'pim': sum(s.get('pim', 0) for s in stats),
#             'powerPlayGoals': sum(s.get('powerPlayGoals', 0) for s in stats),
#             'shorthandedGoals': sum(s.get('shorthandedGoals', 0) for s in stats),
#             'gameWinningGoals': sum(s.get('gameWinningGoals', 0) for s in stats),
#             'shots': sum(s.get('shots', 0) for s in stats),
#         }
#         return aggregated
    
#     # Get aggregated stats for both players
#     agg_stats_a = aggregate_season_stats(player_a_data, player_a_game_type_id.value)
#     agg_stats_b = aggregate_season_stats(player_b_data, player_b_game_type_id.value)
    
#     # Create comparison table
#     comparison_data = []
    
#     if agg_stats_a or agg_stats_b:
#         stat_fields = ['gamesPlayed', 'goals', 'assists', 'points', 'pim', 
#                       'powerPlayGoals', 'shorthandedGoals', 'gameWinningGoals', 'shots']
        
#         for field in stat_fields:
#             row = {'Stat': field}
#             if agg_stats_a:
#                 row['Player A (All Seasons)'] = agg_stats_a.get(field, '-')
#             if agg_stats_b:
#                 row['Player B (All Seasons)'] = agg_stats_b.get(field, '-')
#             comparison_data.append(row)
        
#         df_comparison = pd.DataFrame(comparison_data)
#         display(df_comparison.to_html(escape=False, index=False), HTML('<br>'))
#     else:
#         print("No aggregated stats found.")

# # Display button
# display_agg_season_button = widgets.Button(
#     description='Display Aggregated Season Totals',
#     button_style='success'
# )

# agg_season_output = widgets.Output()

# def on_display_agg_season_click(b):
#     agg_season_output.clear_output()
#     with agg_season_output:
#         display_aggregated_season_totals()

# display_agg_season_button.on_click(on_display_agg_season_click)

# display(
#     VBox([
#         display_agg_season_button,
#         agg_season_output
#     ])
# )

## Career Totals Comparison

In [ ]:
# def display_career_totals():
#     if not player_a_data and not player_b_data:
#         print("Please search for players first.")
#         return
    
#     # Helper function to get career totals
#     def get_career_totals(player_data):
#         if not player_data or 'careerTotals' not in player_data:
#             return None
#         return player_data['careerTotals']
    
#     career_a = get_career_totals(player_a_data)
#     career_b = get_career_totals(player_b_data)
    
#     if not career_a and not career_b:
#         print("No career totals found.")
#         return
    
#     # Display Regular Season Career Totals
#     print("\n=== REGULAR SEASON CAREER TOTALS ===")
    
#     reg_season_data = []
#     if career_a and 'regularSeason' in career_a:
#         reg_a = career_a['regularSeason']
#         stat_fields = ['gamesPlayed', 'goals', 'assists', 'points', 'plusMinus', 'pim',
#                       'powerPlayGoals', 'shorthandedGoals', 'gameWinningGoals', 'shots', 'shootingPctg']
        
#         for field in stat_fields:
#             row = {'Stat': field}
#             row['Player A'] = reg_a.get(field, '-')
#             if career_b and 'regularSeason' in career_b:
#                 reg_b = career_b['regularSeason']
#                 row['Player B'] = reg_b.get(field, '-')
#             reg_season_data.append(row)
    
#     if reg_season_data:
#         df_reg = pd.DataFrame(reg_season_data)
#         display(df_reg.to_html(escape=False, index=False), HTML('<br>'))
    
#     # Display Playoffs Career Totals
#     print("\n=== PLAYOFFS CAREER TOTALS ===")
    
#     playoff_data = []
#     if career_a and 'playoffs' in career_a:
#         playoff_a = career_a['playoffs']
#         stat_fields = ['gamesPlayed', 'goals', 'assists', 'points', 'plusMinus', 'pim',
#                       'powerPlayGoals', 'shorthandedGoals', 'gameWinningGoals', 'shots', 'shootingPctg']
        
#         for field in stat_fields:
#             row = {'Stat': field}
#             row['Player A'] = playoff_a.get(field, '-')
#             if career_b and 'playoffs' in career_b:
#                 playoff_b = career_b['playoffs']
#                 row['Player B'] = playoff_b.get(field, '-')
#             playoff_data.append(row)
    
#     if playoff_data:
#         df_playoff = pd.DataFrame(playoff_data)
#         display(df_playoff.to_html(escape=False, index=False), HTML('<br>'))

# # Display button
# display_career_button = widgets.Button(
#     description='Display Career Totals',
#     button_style='success'
# )

# career_output = widgets.Output()

# def on_display_career_click(b):
#     career_output.clear_output()
#     with career_output:
#         display_career_totals()

# display_career_button.on_click(on_display_career_click)

# display(
#     VBox([
#         display_career_button,
#         career_output
#     ])
# )